In [ ]:
from flask import Flask, render_template, request, jsonify
from openai import OpenAI
from dotenv import load_dotenv
import os

load_dotenv()
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

app = Flask(__name__)

@app.route("/")
def home():
    return render_template("index.html")

@app.route("/chat", methods=["POST"])
def chat():
    user_msg = request.json["message"]

    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "user", "content": user_msg}
        ]
    )

    bot_reply = response.choices[0].message["content"]
    return jsonify({"reply": bot_reply})

if __name__ == "__main__":
    app.run(debug=True)


In [ ]:
<!DOCTYPE html>
<html>
<head>
    <title>OpenAI Chat App</title>
    <style>
        body { font-family: Arial; margin: 20px; }
        #chatbox { width: 100%; height: 400px; border:1px solid #ccc; padding:10px; overflow-y:scroll; }
        #input { width: 80%; padding: 10px; }
        #send { padding: 10px; }
    </style>
</head>
<body>
    <h2>OpenAI Chat App</h2>
    <div id="chatbox"></div><br>

    <input id="input" placeholder="Type your message..." />
    <button id="send">Send</button>

    <script>
        const chatbox = document.getElementById("chatbox");

        document.getElementById("send").onclick = async () => {
            let message = document.getElementById("input").value;
            chatbox.innerHTML += `<p><b>You:</b> ${message}</p>`;

            const res = await fetch("/chat", {
                method: "POST",
                headers: { "Content-Type": "application/json" },
                body: JSON.stringify({ message })
            });

            const data = await res.json();
            chatbox.innerHTML += `<p><b>Bot:</b> ${data.reply}</p>`;
            document.getElementById("input").value = "";
            chatbox.scrollTop = chatbox.scrollHeight;
        };
    </script>
</body>
</html>
